# Dicas de tipo (*type hints*)

## Conceitos básicos

Conforme vimos, Python é uma linguagem onde variáveis guardam referências para objetos, e os tipos são associados com os objetos, e não com as variáveis. Da mesma forma, uma função pode retornar objetos de diversos tipos.

In [ ]:
x = 1
print(type(x))
x = "Oi"
print(type(x))

In [ ]:
def f(n):
    if n < 0:
        return "Negativo"
    if n > 0:
        return 1.0
    return 0

print(type(f(-1.0)))
print(type(f(43.7)))
print(type(f(0)))

Essa flexibilidade, apesar de útil em algumas situações, é raramente usada, e muitas vezes confusa. Devido a isso, variáveis, parâmetros de função e retornos de função geralmente têm um tipo bem definido.

Recentemente, Python introduziu a possibilidade de indicar os tipos esperados para variáveis ou valores de retorno de funções, nas chamadas **dicas de tipo**.

Por exemplo, se fazemos:

In [ ]:
a : int = 0

estamos não apenas criando uma variável de nome `a` que tem uma referência para um objeto `int` de valor 0, como também estamos indicando que essa variável irá guardas sempre referências para valores `int`.

Da mesma forma, podemos anotar parâmetros de funções e seus valores de retorno:

In [ ]:
def sum_squares(n: int) -> int:
    return sum(i ** 2 for i in range(n))

Essas anotações indicam que `sum_squares` espera que seu parâmetro `n` seja um `int` e também que ela irá retornar um `int`.

In [ ]:
sum_squares(10)

## Efeito

É importante notar que, para o Python, a anotação de dica de tipo é ignorada, Isto é, ela não tem nenhum efeito durante a execução:

In [ ]:
i : int = 0
print(type(i))
i = 3.2
print(type(i))

De fato, dicas de tipo foram introduzidas como uma forma de **documentação** que pode ser lida por ferramentas especialmente preparadas, como `mypy`, `pyright` e alguns ambientes de programação, que podem usar essas informações para ajudar a encontrar erros e fornecer informações para os usuários.

Note que o Jupyter incorpora as informações de tipo na documentação:

In [ ]:
sum_squares?

## Alguns casos mais complexos

Para tipos simples, como nos exemplos anteriores, basta usar o nome do tipo. Mas em alguns casos isso não é suficiente. Por exemplo, no caso de listas. Não adianta muito saber que temos uma lista se não sabemos do que é essa lista (quer dizer, que tipo de objetos temos na lista).

Para isso usamos uma sintaxe como no exemplo abaixo:

In [ ]:
def only_even(values: list[int]) -> list[int]:
    return list(filter(lambda x: x % 2 == 0, values))

In [ ]:
only_even([1, 2, 3, 4, 5, 6, 7, 8, 9])

Neste exemplo, estamos dizendo que a função `only_even` espera receber em seu parâmetro `values` uma lista de valores do tipo `int`, e irá por sua vez retornar uma lista de valores do tipo `int`.

Para tuplas, a sintaxe é similar:

In [ ]:
def distance(p1: tuple[float, float], p2: tuple[float, float]) -> float:
    from math import hypot
    return hypot(p1[0] - p2[0], p1[1] - p2[1])

In [ ]:
distance((1.0, 0.0), (0.0, 1.0))

Neste exemplo, dizemos que os parâmetros `p1` e `p2` devem ser tuplas com dois valores `float` e a função irá retornar um `float`.

Também dicionários têm uma sintaxe semelhante, com indicação do tipo da chave e do tipo do valor:

In [ ]:
def square_roots_of(values: list[int]) -> dict[int, float]:
    from math import sqrt
    return {i: sqrt(i) for i in values}

In [ ]:
square_roots_of([1, 2, 4, 6, 8, 3, 9, 2, 4, 3])

Este exemplo indica que a função `square_roots_of` retorna um dicionário no qual as chaves são de tipo `int` e os valores de tipo `float`.

Em algumas situações, queremos que uma variável possa referenciar mais do que um tipo, ou uma função retornar mais do que um tipo. Se temos uma lista pequena de tipos possíveis (o caso normal) podemos indicar isso pelo uso do operador `|`:

In [ ]:
def remember_value(x: int, old_values: list[int] | None = None) -> list[int]:
    if old_values is None:
        old_values = []
    old_values.append(x)
    return old_values

In [ ]:
allv = remember_value(10)
print(allv)
allv = remember_value(12, allv)
print(allv)

Neste caso, dizemos que o parâmetro `old_values` pode ou ter uma referência para uma lista de `int` ou então ser `None`.

Este exemplo também demonstra que o valor default é colocado depois da dica de tipo.

## Classes

Seus tipos definidos como classes podem também ser usados nas dicas de tipos, e o sistema respeita herança, isto é, se um parâmetro (ou outra variável) é especificado como do tipo da classe `A`, então você pode passar referências para objetos de classes derivadas de `A`.

In [ ]:
class Counter:
    def __init__(self):
        self._value = 0
        
    def get(self):
        return self._value
    
    def up(self):
        self._value += 1
        
class UpDownCounter(Counter):
    def __init__(self):
        super().__init__()
        
    def down(self):
        self._value -= 1

In [ ]:
up_1 : Counter = Counter()
down_1 : UpDownCounter = UpDownCounter()

In [ ]:
def up_all(counters: list[Counter]) -> None:
    for c in counters:
        c.up()

In [ ]:
print(up_1.get())
print(down_1.get())

up_all([up_1, down_1])

print(up_1.get())
print(down_1.get())

O `-> None` na definição da função indica que essa função não retorna nada.

In [ ]:
up_all?

## Sinônimos de tipos

Para melhorar a documentação, podemos definir *sinônimos* para alguns tipos. Isso permite maior clareza na compreensão do código, quando usado corretamente.

Por exemplo, a função `distance` definida anteriormente e repetida abaixo não é muito clara, pois não sabemos o que cada tupla representa:

In [ ]:
def distance(p1: tuple[float, float], p2: tuple[float, float]) -> float:
    from math import hypot
    return hypot(p1[0] - p2[0], p1[1] - p2[1])

Podemos melhorar a clareza definido um nome para o tipo que representa um ponto em um plano bidimensional:

In [ ]:
Point2D = tuple[float, float]

def distance(p1: Point2D, p2: Point2D) -> float:
    from math import hypot
    return hypot(p1[0] - p2[0], p1[1] - p2[1])

A leitura do código é agora mais clara. O problema é que isso não ajuda na documentação:

In [ ]:
distance?

## O módulo `typing`

Para algumas situações, o que foi apresentado acima não é suficiente para anotar os tipos de um código Python. O módulo `typing` foi criado em parte para permitir a anotação de tipos em situações que apenas o uso de tipos pré-definidos ou criados pelo usuário não é suficiente, e em parte para permitir uma melhor documentação dos tipos.

Vejamos algumas possibilidades fornecidas por esse módulo.

### Definição de novos tipos

Podemos definir um novo tipo, tanto dando novo nome a tipo já existente quanto dando um nome a um tipo composto.

In [ ]:
from typing import NewType

Index = NewType('Index', int)
Vector = NewType('Vector', list[float])

def get_range(vec: Vector, start: Index, finish: Index) -> Vector:
    return vec[start:finish]

get_range([10., 20., 30., 40., 50., 60., 70.], 2, 5)

Agora temos uma boa documentação:

In [ ]:
get_range?

Podemos fazer o mesmo com o tipo `Point2D` acima:

In [ ]:
Point2D = NewType('Point2D', tuple[float, float])

def distance(p1: Point2D, p2: Point2D) -> float:
    from math import hypot
    return hypot(p1[0] - p2[0], p1[1] - p2[1])

In [ ]:
distance?

### Valores opcionais

Um caso comum é quando uma variável ou o retorno de uma função pode ter um valor de um dado tipo ou então ter `None`, quando nenhum valor ainda foi fornecido ou não pode ser calculado.

Vimos que isso pode ser expresso com o uso do operador `|`, mas uma forma mais clara é usar `Optional`:

In [ ]:
from typing import Optional

def remember_value(x: int, old_values: Optional[list[int]] = None) -> list[int]:
    if old_values is None:
        old_values = []
    old_values.append(x)
    return old_values

Esse código diz que o parâmetro `old_values` é opcional, podendo ser ou uma lista de inteiros ou `None`.

In [ ]:
def quotient_remainder(n: int, d: int) -> Optional[tuple[int, int]]:
    if d == 0:
        return None
    return n // d, n % d

In [ ]:
print(quotient_remainder(10, 3))
print(quotient_remainder(12, 0))

### Variáveis de tipo

Operações definidas por funções são genéricas, e muitas vezes queremos deixar essa genericidade explícita. Por exemplo, a função abaixo retorna o último elemento de uma lista:

In [ ]:
def last(values):
    return values[-1]

Obviamente, a função funciona não importa o tipo de elementos da lista, mas o valor retornado vai ser do tipo de elementos contidos na lista.

Assim, se anotamos algo como:

In [ ]:
def last(values: list[int]) -> int:
    return values[-1]

Estamos sendo muito restritivos. Mas como podemos dizer que o tipo do retorno da fução é o tipo dos elementos da lista? Para isso, podemos definir uma *variável de tipo*, que é usada para indicar um tipo qualquer, mas uma vez escolhido o tipo em um contexto, o mesmo tipo é usado em todo lugar.

In [ ]:
from typing import TypeVar

T = TypeVar('T')

def last(values: list[T]) -> T:
    return values[-1]

In [ ]:
last?

Isso pode ser usado também para indicar que vários parâmetros são do mesmo tipo:

In [ ]:
def shift(a: T, b: T, c: T) -> tuple[T, T, T]:
    return b, c, a

In [ ]:
shift(1, 2, 3)

In [ ]:
shift?

Neste código, queremos que os três parâmetros sejam referências para objetos do mesmo tipo.

### Funções

Como vimos, Python permite que (referências para) funções sejam colocadas em variáveis ou retornadas por funções. Para descrever o tipo dessas funções, usamos `Callable` do módulo `typing`.

Por exemplo, a função abaixo soma os valores detornados por uma dada função aplicada a todos os valores de uma lista:

In [ ]:
def total_map(f, values):
    return sum(map(f, values))

Como podemos anotar essa função? Primeiro, precisamos saber como descrever o tipo do parâmetro `f` e depois do valor de retorno. Vamos por enquanto supor que `values` é uma lista de valores, todos do mesmo tipo. Vamos usar a variável de tipo `T` previamente definida para indicar o tipo dos elementos nessa lista.

O tipo de `f` é declarado usando `Callable`, que tem dois "parâmetros": uma lista com os tipos dos parâmetros da função e o tipo do valor retornado.

In [ ]:
from typing import Callable

def total_map(f: Callable[[T], T], values: list[T]) -> T:
    return sum(map(f, values))

In [ ]:
total_map?

Esse código diz que:
- `f` é uma função (ou alguma outra coisa que pode ser chamada como uma função, por exemplo um objeto de uma classe que implementa o método `__call__`) que recebe um parâmetro de um certo tipo e retorna um valor do mesmo tipo do parâmetro recebido;
- `values` é uma lista de objetos do mesmo tipo do parâmetro da função;
- O valor retornado é do mesmo tipo dos elementos da lista.

Agora vamos ver um exemplo de uma função que retorna uma função.

In [ ]:
def scale_by(a, b):
    def do_it(x):
        return a * x + b
    return do_it

Essa função retorna uma função que recebe um parâmetro e o multiplica por `a` e depois soma `b`.

In [ ]:
f1 = scale_by(2, 3)
print(f1(4), f1(7))
f2 = scale_by(0.1, -2)
print(f2(4), f2(7))

Vamos anotar novamente fazendo uso da variável de tipo `T`, para permitir que a função seja usada para diversos tipos de dados.

In [ ]:
def scale_by(a: T, b: T) -> Callable[[T], T]:
    def do_it(x: T) -> T:
        return a * x + b
    return do_it

**Nota:** Neste exemplo, estamos dizendo que a função funciona para qualquer tipo, desde que os tipos de `a`, `b` e do parâmetro `x` a ser fornecido para a função criada sejam o mesmo. Isso não é verdade, pois precisamos ter os operadores de prdouto e soma definidos.

### Classes genéricas

Da mesma forma que funções, classes também podem ser genéricas, no sentido que funcionam para vários tipois de dados.

Por exemplo, a classe (mais ou menos inútil) abaixo, guarda os últimos 10 valores fornecidos.

In [ ]:
class LastTen:
    def __init__(self):
        self._values = []
        
    def insert(self, x):
        self._values.append(x)
        if len(self._values) == 11:
            self._values.pop(0)
    
    def get(self):
        return self._values[:]

In [ ]:
x = LastTen()
for i in range(1, 100, 3):
    x.insert(i)
print(x.get())

Obviamente, essa classe funciona para outros tipos de dados também:

In [ ]:
y = LastTen()
for i in range(1, 100, 3):
    y.insert(i / 2)
print(y.get())

Agora, suponha que queremos deixar claro que desejamos que todos os lementos da lista sejam do mesmo tipo, mas aceitamos qualquer tipo na lista. Podemos redefinir a classe usando dicas de tipo com o auxílio de `Generic` (e da nossa variável de tipo `T` já definida):

In [ ]:
from typing import Generic

class LastTen(Generic[T]):
    def __init__(self) -> None:
        self._values: list[T] = []
        
    def insert(self, x: T) -> None:
        self._values.append(x)
        if len(self._values) == 11:
            self._values.pop(0)
    
    def get(self) -> list[T]:
        return self._values[:]

Isso nos diz que:
- `LastTen` é uma classe genérica, que se adapta a um tipo genérico denominado `T`.
- O método `insert` aceita um parâmetro do tipo `T` e não retorna nada.
- O método `get` retorna uma lista de `T`.

Suponhamos que fazemos uma função para escolher aleatoriamente um dos últimos 10 elementos armazenados. A versão anotada será:

In [ ]:
def choose_one_recent(options: LastTen[T]) -> T:
    from random import choice
    return choice(options.get())

In [ ]:
choose_one_recent(y)

In [ ]:
choose_one_recent(x)

In [ ]:
choose_one_recent?

Aqui indicamos que `choose_one_recent` recebe um objeto com os 10 mais recentes de algum tipo `T` e retorna um objeto desse tipo. Note como agora usamos a variável de tipo `T` na nossa classe para indicar qual o tipo de valores que são guardados pelo objeto da classe.

### Qualquer tipo

Frequentemente, queremos reverter para o caso normal de Python, onde uma referência pode ser para objetos de qualquer tipo. Para isso, temos `Any`.

Por exemplo, se quisermos uma versão de `LastTen` que aceita tipos misturados nos objetos guardados, podemos usar `Any`:

In [ ]:
from typing import Any

class LastTenMixed:
    def __init__(self) -> None:
        self._values: list[Any] = []
        
    def insert(self, x: Any) -> None:
        self._values.append(x)
        if len(self._values) == 11:
            self._values.pop(0)
    
    def get(self) -> list[Any]:
        return self._values[:]

In [ ]:
z = LastTenMixed()
for o in [1, 2.0, 3.0 + 4.0j, 'cinco']:
    z.insert(o)
print(z.get())

Veja como neste caso não precisamos fazer a classe derivar de `Generic`, pois não existe necessidade de manter consistência de tipos.

O módulo `typing` tem vários outros elementos, especialmente para lidar com problemas mais complexos.

Veja a [documentação](https://docs.python.org/3/library/typing.html).

## O módulo `collections.abc`

Para permitir que funções possam ser anotadas de forma não restritiva, é necessário especificar apenas o mínimo necessário sobre o que o tipo do parâmetro precisa.

O módulo `collections.abc` apresenta alguns elementos para ajudar em situações frquentemente encontradas.

Para entender do que se trata isso, vejamos novamente a função `only_even` definida acima:

In [ ]:
def only_even(values: list[int]) -> list[int]:
    return list(filter(lambda x: x % 2 == 0, values))

A especificação apresentada para o parâmetro `values` é muito restritiva, pois o código não funciona apenas para uma lista de `int`. O conceito de números pares somente se aplica a inteiros, então o uso de `int` está correto, mas os valores inteiros não precisam estar numa lista. Por exemplo, podemos fazer:

In [ ]:
only_even(range(12))

O que precisamos para o parâmetro `values` é que ele nos forneça uma sequência de valores inteiros. Isto é, ele precisa ser **iterável**.

Para isso, podemos usar `Iterable` do módulo `collections.abc`.

In [ ]:
from collections.abc import Iterable

def only_even(values: Iterable[int]) -> list[int]:
    return list(filter(lambda x: x % 2 == 0, values))

In [ ]:
only_even?

Os "tipos" definidos em `collections.abc` são caraterizados pela presença de métodos específicos. Por exemplo, para ser um `Iterable` o tipo tem que implementar o método `__iter__`.

Seguem alguns tipos adicionais (veja mais na [documentação](https://docs.python.org/3/library/collections.abc.html).

| Tipo | Métodos |
|------|---------|
| `Container` | `__contains__` |
| `Iterator` | `__iter__` e `__next__` |
| `Sized` | `__len__` |
| `Collection` | `__contains__`, `__iter__` e `__next__` |
| `Sequence` | `__contains__`, `__iter__`, `__next__`, `__getitem__`, `__reversed__`, `index`, `count` |
| `MutableSequnce` | mesmos de `Sequence` mais `__setitem__`, `__delitem__`, `insert`, `append`, `reverse`, `extend`, `pop`, `remove`, `__iadd__` |

Vamos rever a função `last` anteriormente definida (cópia abaixo):

In [ ]:
def last(values: list[int]) -> int:
    return values[-1]

Novamente, para o que essa função faz, não é necessário:
- Que os elementos de `values` sejam `int`. Eles podem ser de qualquer tipo, inclusive misturados.
- Não é necessário que os elementos estam numa lista. Eles precisam apenas estar em algo que possamos indexar.
- Não estamos alterando o contéiner de valores fornecido, então não precisamos que o objeto `values` seja mutável.

Portanto, podemos reescrever as anotações de tipo:

In [ ]:
from collections.abc import Sequence

def last(values: Sequence[Any]) -> Any:
    return values[-1]

In [ ]:
last?

# Exercícios

Faça anotações de tipo adequadas para as seguintes funções:

### 1

In [ ]:
from math import cos

def f(x):
    return cos(x) ** 2 - 1

### 2

In [ ]:
def drop_extension(name):
    iext = name.rfind('.')
    return name[:iext] if iext >= 0 else name

### 3

In [ ]:
def parse(s):
    tokens = s.split(',')
    tokens = [t.strip() for t in tokens]
    all_pairs = {}
    for i in range(0, len(tokens), 2):
        all_pairs[tokens[i]] = tokens[i+1]
    return all_pairs

### 4

In [ ]:
def max_abs(values):
    return max(abs(x) for x in values)